In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# Step 1: Connect to the PostgreSQL database


In [3]:
db_user = 'postgres'
db_password = 'postgres'
db_host = 'localhost'
db_port = '5432'
db_name = 'access'

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')



# Step 2: Query necessary data from the database


In [4]:
# Get courses of interest
courses_query = """
SELECT id, slug AS moduleslug
FROM course
WHERE slug IN ('fs24-sta120', 'fs24-mat183')
"""
courses_df = pd.read_sql_query(courses_query, engine)

# Rename moduleslug to moduleSlug
courses_df.rename(columns={'moduleslug': 'moduleSlug'}, inplace=True)
print(courses_df)


   id   moduleSlug
0   2  fs24-sta120
1   3  fs24-mat183


In [129]:

# Get assignments for the selected courses
assignments_query = """
SELECT *
FROM assignment
WHERE assignment.course_id IN (SELECT id FROM course WHERE slug IN ('fs24-sta120', 'fs24-mat183'))
"""
assignments_df = pd.read_sql_query(assignments_query, engine)

print(assignments_df)


     id  enabled            end_date  ordinal_num           slug  \
0   402     True 2024-03-05 23:59:00            1   week1_mat183   
1   352     True 2024-03-04 23:45:00            2   week2_sta120   
2   302     True 2024-03-04 23:45:00            1   week1_sta120   
3   452     True 2024-03-11 23:45:00            3   week3_sta120   
4   453     True 2024-03-12 23:59:00            2   week2_mat183   
5   502     True 2024-03-19 23:59:00            3   week3_mat183   
6   503     True 2024-03-18 23:45:00            4   week4_sta120   
7   552     True 2024-03-25 23:45:00            5   week5_sta120   
8   553     True 2024-03-26 23:59:00            4   week4_mat183   
9   602     True 2024-04-09 23:59:00            5   week5_mat183   
10  603     True 2024-04-08 23:45:00            6   week6_sta120   
11  604     True 2024-04-16 23:59:00            6   week6_mat183   
12  605     True 2024-04-15 23:45:00            7   week7_sta120   
13  652     True 2024-04-22 23:45:00            

In [137]:
# Get tasks for the selected assignments
tasks_query = """
SELECT
    task.id,
    task.slug,
    task.assignment_id,
    task.max_points,
    solution_file.id AS solution_file_id,
    solution_file.name AS solution_file_name,
    solution_file.mime_type AS solution_file_mime_type,
    solution_file.template AS solution,
    instruction_file.id AS instruction_file_id,
    instruction_file.name AS instruction_file_name,
    instruction_file.mime_type AS instruction_file_mime_type,
    instruction_file.template AS instruction,
    course.slug AS course_slug
FROM task
LEFT JOIN task_file AS solution_file 
    ON task.id = solution_file.task_id AND solution_file.solution = TRUE
LEFT JOIN task_file AS instruction_file 
    ON task.id = instruction_file.task_id AND instruction_file.instruction = TRUE
LEFT JOIN assignment ON task.assignment_id = assignment.id
LEFT JOIN course ON assignment.course_id = course.id
WHERE task.assignment_id IN (
    SELECT id 
    FROM assignment 
    WHERE course_id IN (
        SELECT id 
        FROM course 
        WHERE slug IN ('fs24-sta120', 'fs24-mat183')
    )
)

"""
tasks_df = pd.read_sql_query(tasks_query, engine)
print(tasks_df.head())

    id            slug  assignment_id  max_points  solution_file_id  \
0  402  1_week1_sta120            302         1.0               606   
1  403  2_week1_sta120            302         1.0               611   
2  753  2_week8_sta120            652         1.0              1261   
3  709  3_week6_sta120            603         1.0              1146   
4  404  3_week1_sta120            302         1.0               617   

  solution_file_name solution_file_mime_type  \
0           script.R              text/plain   
1           script.R              text/plain   
2           script.R              text/plain   
3           script.R              text/plain   
4           script.R              text/plain   

                                            solution  instruction_file_id  \
0  png(file = "solution.png")\nrequire(fields) # ...                  602   
1                                 demo(persp) \n\n\n                  607   
2   png(file="solution.png")\nmu <- c(1, 2)\n sig... 

In [138]:
print(len(tasks_df))

168


In [139]:
# Get user entities (students)
users_query = """
SELECT id AS user_id, email, username
FROM user_entity
"""
users_df = pd.read_sql_query(users_query, engine)
print(users_df.head())

                                user_id                 email  \
0  d95603c0-5a0f-4c9a-85f2-f2036892966d        student@uzh.ch   
1  a08fc0d2-854a-4860-9f12-6c06b17df93e     supervisor@uzh.ch   
2  c3b197c2-6295-4b30-a764-d06d6935f9d5  shubhi.pareek@uzh.ch   
3  b2e59824-54a6-4dd2-aff5-b1c87c04b2f9                  None   
4  7a5d43de-f501-4a44-9edb-6fa564804984                  None   

                           username  
0                    student@uzh.ch  
1                 supervisor@uzh.ch  
2              shubhi.pareek@uzh.ch  
3  service-account-realm-management  
4                             admin  


In [140]:
# Get submissions for the selected tasks with submission file content and grade
submissions_query = """
SELECT 
    submission.id,
    submission.points,
    submission.valid,
    submission.evaluation_id,
    submission.user_id,
    evaluation.best_score,
    submission_file.content,
    evaluation.task_id
FROM submission
LEFT JOIN evaluation ON submission.evaluation_id = evaluation.id
LEFT JOIN submission_file ON submission.id = submission_file.submission_id
WHERE submission.command = 'GRADE'
  AND evaluation.task_id IN (
      SELECT id 
      FROM task 
      WHERE assignment_id IN (
          SELECT id 
          FROM assignment 
          WHERE course_id IN (
              SELECT id 
              FROM course 
              WHERE slug IN ('fs24-sta120', 'fs24-mat183')
          )
      )
  )

""" 


submissions_df = pd.read_sql_query(submissions_query, engine)
print(submissions_df.head())

       id  points  valid  evaluation_id                      user_id  \
0  357735     1.0   True          49202  julianpatrick.stoerr@uzh.ch   
1  357739     0.0   True          49204            floxmas@gmail.com   
2  357741     1.0   True          49203  julianpatrick.stoerr@uzh.ch   
3  350328     0.0   True          47315  anjana.thatheeskumar@uzh.ch   
4  350330     1.0   True          47315  anjana.thatheeskumar@uzh.ch   

   best_score                                            content  task_id  
0         1.0  png(file = "solution.png")\n\n## Uniforme Vert...      761  
1         1.0  png(file = "solution.png")\n\n## Normalverteil...      762  
2         1.0  png(file = "solution.png")\n\n## Normalverteil...      762  
3         1.0  png(file = "solution.png")\n\n## Normalverteil...      762  
4         1.0  png(file = "solution.png")\n\n## Normalverteil...      762  


In [141]:
print(len(submissions_df))

55289


# Step 4: Save Data

In [143]:
tasks_df.to_csv('data/tasks_df.csv', index=False)
submissions_df.to_csv('data/submissions_df.csv', index=False)
users_df.to_csv('data/users_df.csv', index=False)
